## Kalman Filter
This is the implementation of constant turn model. Consider a situation where our object under observation is moving in a circular path at constant velocity. The positions of the object are measured at each timestamp through the camera sensors. Lets begin with defining the state space vector of our object 

### Observation Model

$$
z_k = \textbf{H}.x_k +v_k 
$$

$$
z_k = \begin{bmatrix}
    1&0&0&0  \\
    0&0&1&0 \\
\end{bmatrix}. x_k
$$ 

### Error Covariance
$$ w_k \sim \mathcal{N}(0,\,\textbf{Q})$$

$$\textbf{Q} = S \begin{bmatrix}
    σ_x^2&0&σ_{x \Delta x}&0  \\
    0&σ_y^2&0&σ_{y \Delta y}\\
    σ_{\Delta x x}&0&σ_{\Delta x}^2&0 \\
    0&σ_{\Delta y y}&0&σ_{\Delta y}^2 \\
\end{bmatrix}$$

$$\textbf{Q} = S \begin{bmatrix}
    \frac{T^3}{3}&\frac{T^2}{2}&0&0  \\
    \frac{T^2}{2}&T&0&0\\
    0&0&\frac{T^3}{3}&\frac{T^2}{2} \\
    0&0&\frac{T^2}{2}&T\\
\end{bmatrix}$$




### Measurement Covariance
$$ v_k \sim \mathcal{N}(0,\,\textbf{R})$$
$$ \textbf{R} = σ^2 \textbf{I}$$
$$\textbf{R} =  \begin{bmatrix}
    σ_x^2&0  \\
    0&σ_y^2\\
\end{bmatrix}$$

### Initial Uncertainity 
$$\textbf{P} = \begin{bmatrix}
    σ_x^2&0&0&0  \\
    0&σ_{Δx}^2&0&0 \\
    0&0&σ_x^2&0 \\
    0&0&0&σ_{Δy}^2 \\
\end{bmatrix} $$
 




### Prediction Step
$${x_{k+1}} = A.{x_k} 
$$

$$
P_{k+1} = A_k.P_k.A^T +Q
$$

### Update Step

For each measurement from camera:

Computing the Kalman Gain:
$$
K_k = P_k.H^T(H.P_k.H^T+R)^{-1} 
$$

Updating the estimate using position measurements from the camera:
$$
x_k = x_k+K_k(z_x-H.x_k)
$$

Updating the error covariance matrix:
$$
P_k = (I-K_k.H)P_k
$$
